In [58]:
import requests
import pandas as pd

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [46]:
url = 'https://api.foursquare.com/v3/places/nearby'
API_key = 'fsq3dQYQp2ka71CYVbI6ehUtmigysRY/oygrq7j7KFRHyI8='

#Set location parameters
latitude = '43.651070'
longitude = '-79.347015'

In [47]:
params = {
    'll': f'{latitude},{longitude}',
    'radius': 1000, #in meter
    'query': 'restaurant'
}
headers = {"Authorization": API_key
}

In [48]:
#Send API request and parse JSON response
response = requests.get(url, params=params, headers=headers)

print(response.status_code)

200


In [57]:
data = response.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [63]:
# Extract the list of restaurants
restaurants = data['results']

In [68]:
# Iterate over each restaurant and extract the details
for restaurant in restaurants:
    name = restaurant['name']
    location = restaurant['location']['formatted_address']
    latitude = restaurant['geocodes']['main']['latitude']
    longitude = restaurant['geocodes']['main']['longitude']

    # Print the details
    print(f"Name: {name}")
    print(f"Location: {location}")
    print(f'latitude: {latitude}')
    print(f'longitude: {longitude}')
    print()

Name: est Restaurant
Location: 729 Queen St E, Toronto ON M4M 1H1
latitude: 43.658791
longitude: -79.349195

Name: Restaurant 20 Victoria
Location: 20 Victoria St, Toronto ON M8V 1M6
latitude: 43.650025
longitude: -79.377257

Name: Lan Restaurant
Location: 755 Queen St E, Toronto ON M4M 1H3
latitude: 43.6591
longitude: -79.348029

Name: Gio Rana's Really Really Nice Restaurant
Location: 1220 Queen St E, Toronto ON M4M 1L7
latitude: 43.663345
longitude: -79.330418

Name: P.J. O'Brien Irish Pub and Restuarant
Location: 39 Colborne St (at Leader Ln.), Toronto ON M5E 1E3
latitude: 43.648892
longitude: -79.375067

Name: The Real Jerk Restaurant
Location: 842 Gerrard St E (Carlaw Avenue), Toronto ON M4M 1Y7
latitude: 43.667754
longitude: -79.343227

Name: Hot House Cafe
Location: 35 Church St (at Front St E), Toronto ON M5E 1T3
latitude: 43.648927
longitude: -79.373811



Put your parsed results into a DataFrame

In [70]:
# Create a list to store the restaurant details
rest_details = []

In [71]:
# Iterate over each restaurant and extract the details
for restaurant in restaurants:
    name = restaurant['name']
    location = restaurant['location']['formatted_address']
    latitude = restaurant['geocodes']['main']['latitude']
    longitude = restaurant['geocodes']['main']['longitude']

    # Append the restaurant details as a dictionary to the list
    rest_details.append({
        'Retaurant name': name,
        'Name': name,
        'Latitude': latitude,
        'Longitude': longitude
    })

In [72]:
# Create the bike stations DataFrame
fq_df = pd.DataFrame(rest_details)
fq_df

,Retaurant name,Name,Latitude,Longitude
0,est Restaurant,est Restaurant,43.658791,-79.349195
1,Restaurant 20 Victoria,Restaurant 20 Victoria,43.650025,-79.377257
2,Lan Restaurant,Lan Restaurant,43.659100,-79.348029
3,Gio Rana's Really Really Nice Restaurant,Gio Rana's Really Really Nice Restaurant,43.663345,-79.330418
4,P.J. O'Brien Irish Pub and Restuarant,P.J. O'Brien Irish Pub and Restuarant,43.648892,-79.375067
5,The Real Jerk Restaurant,The Real Jerk Restaurant,43.667754,-79.343227
6,Hot House Cafe,Hot House Cafe,43.648927,-79.373811


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating